In [46]:
import time
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [47]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [86]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [87]:
dataset.head(10)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0.0,0.0,NaN,1.488361

In [88]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [89]:
dataset.columns

Index(['id', 'time', 'activityrecognition#0', 'activityrecognition#1',
       'android.sensor.accelerometer#mean', 'android.sensor.accelerometer#min',
       'android.sensor.accelerometer#max', 'android.sensor.accelerometer#std',
       'android.sensor.game_rotation_vector#mean',
       'android.sensor.game_rotation_vector#min',
       'android.sensor.game_rotation_vector#max',
       'android.sensor.game_rotation_vector#std',
       'android.sensor.gravity#mean', 'android.sensor.gravity#min',
       'android.sensor.gravity#max', 'android.sensor.gravity#std',
       'android.sensor.gyroscope#mean', 'android.sensor.gyroscope#min',
       'android.sensor.gyroscope#max', 'android.sensor.gyroscope#std',
       'android.sensor.gyroscope_uncalibrated#mean',
       'android.sensor.gyroscope_uncalibrated#min',
       'android.sensor.gyroscope_uncalibrated#max',
       'android.sensor.gyroscope_uncalibrated#std',
       'android.sensor.light#mean', 'android.sensor.light#min',
       'android.se

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [90]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5892
Data columns (total 70 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   id                                               5893 non-null   int64  
 1   time                                             5893 non-null   float64
 2   activityrecognition#0                            0 non-null      float64
 3   activityrecognition#1                            5893 non-null   float64
 4   android.sensor.accelerometer#mean                5893 non-null   float64
 5   android.sensor.accelerometer#min                 5893 non-null   float64
 6   android.sensor.accelerometer#max                 5893 non-null   float64
 7   android.sensor.accelerometer#std                 5893 non-null   float64
 8   android.sensor.game_rotation_vector#mean         5094 non-null   float64
 9   android.sensor.game_rotation_v

In [91]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.146752,14.137338,0.081745,15.141369,15.34

In [92]:
dataset.isnull().sum()

id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                       720
android.sensor.gyroscope#min                    

In [93]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-93-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000

In [129]:
dataset = dataset[(['time',
                    'accelerometer_mean',
                    'accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'linear_acceleration_mean',
                    'linear_acceleration_min',
                    'linear_acceleration_max',
                    'linear_acceleration_std',
                    'pressure_mean',
                    'proximity_std',
                    'rotation_vector_mean',
                    'rotation_vector_min',
                    'rotation_vector_max',
                    'rotation_vector_std',
                    'step_counter_mean',
                    'step_counter_max',
                    'step_counter_min',
                    'step_counter_std',
                    'sound_mean',
                    'user',
                    'target'
                 ])]

In [130]:
dataset.head()

,time,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,pressure_mean,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_max,step_counter_min,step_counter_std,sound_mean,user,target
0,78.0,9.811476,9.758895,9.849411,0.014626,0.001651,0.016221,0.020978,0.002495,0.052410,0.011045,1004.090261,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,U12,Still
1,145.0,9.939207,7.707437,17.146631,1.775944,0.036326,0.039023,0.879220,0.641117,1.185810,0.278730,1008.274660,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,U12,Car
2,150.0,9.827178,9.804817,9.849262,0.011199,0.001525,0.016302,0.034110,0.025709,0.042511,0.011881,1005.977540,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,U12,Still
3,105.0,9.673039,7.659674,12.304298,0.862553,0.036440,0.050759,1.488361,0.575910,2.415879,0.920077,1006.950440,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,U12,Car
4,77.0,9.993466,8.965621,10.891645,0.504117,0.183202,0.265652,1.658308,0.379803,2.795995,1.122225,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,U1,Car


In [131]:
# split user = u12 & u3 as test/train
dataset_test = dataset[(dataset['user'] == 'U12')|(dataset['user'] == 'U3')]
dataset_train = dataset[(dataset['user'] != 'U12')&(dataset['user'] != 'U3')]

In [132]:
dataset_train.sample(10)

,time,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,pressure_mean,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_max,step_counter_min,step_counter_std,sound_mean,user,target
2797,35.0,9.460300,9.433216,9.491637,0.018020,0.006483,0.005133,0.028421,0.019285,0.044310,0.009510,1025.7163,NaN,0.668653,0.668455,0.668785,0.000175,NaN,NaN,NaN,NaN,53.423457,U2,Still
3450,103.0,9.579318,7.909174,11.463022,0.981525,0.133029,0.159202,2.174544,1.147978,3.187198,1.019681,NaN,NaN,0.966949,0.965915,0.967762,0.000520,NaN,NaN,NaN,NaN,89.786793,U1,Bus
3476,86.0,9.782870,9.698872,9.882769,0.050857,0.009575,0.040774,0.101067,0.051376,0.150758,0.070274,1058.0823,NaN,0.977172,0.976841,0.977503,0.000468,NaN,NaN,NaN,NaN,68.977217,U7,Train
4156,22.0,9.788703,7.641998,12.805304,0.948175,0.309666,0.507918,0.156560,0.156560,0.156560,0.081404,NaN,NaN,0.324455,0.324455,0.324455,0.007113,NaN,NaN,NaN,NaN,84.167892,U10,Still
2645,156.0,10.265174,10.215210,10.290742,0.021095,0.009573,0.034385,0.021541,0.011465,0.035675,0.009906,NaN,NaN,0.997543,0.997473,0.997649,0.000062,NaN,NaN,NaN,NaN,71.688886,U1,Still
2053,12.0,9.806202,9.712029,9.911716,0.040194,0.011655,0.043361,0.048009,0.036651,0.059368,0.016063,1058.3616,NaN,0.214126,0.214126,0.214126,0.003533,NaN,NaN,NaN,NaN,79.151693,U7,Train
3979,67.0,10.114633,9.120167,11.077028,0.557913,0.052823,0.057919,0.677216,0.362879,0.991552,0.444539,NaN,NaN,0.352846,0.346219,0.360866,0.005234,NaN,NaN,NaN,NaN,89.752947,U1,Car
1942,113.0,9.812863,9.173014,10.659664,0.278183,0.013215,0.059741,0.109549,0.109549,0.109549,0.057502,1030.5078,NaN,0.679122,0.676945,0.681299,0.003079,NaN,NaN,NaN,NaN,85.852574,U7,Train
1876,111.0,10.033485,9.973907,10.128033,0.030392,0.005464,0.256838,0.160798,0.160798,0.160798,0.034184,NaN,NaN,0.475174,0.475046,0.475385,0.000184,6803.0,6803.0,6803.0,NaN,51.222028,U10,Still
804,16.0,10.397750,8.649993,11.105722,0.614978,0.301824,0.300391,1.157853,0.848359,1.761014,0.522412,NaN,NaN,0.628933,0.463903,0.710253,0.098106,NaN,NaN,NaN,NaN,89.797764,U1,Car


In [133]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [134]:
num_vars = ['time',
                    'accelerometer_mean',
                    'accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'linear_acceleration_mean',
                    'linear_acceleration_min',
                    'linear_acceleration_max',
                    'linear_acceleration_std',
                    'pressure_mean',
                    'proximity_std',
                    'rotation_vector_mean',
                    'rotation_vector_min',
                    'rotation_vector_max',
                    'rotation_vector_std',
                    'step_counter_mean',
                    'step_counter_max',
                    'step_counter_min',
                    'step_counter_std',
                  'sound_mean']

In [135]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import QuantileTransformer, StandardScaler
num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputing', KNNImputer(n_neighbors=5))
])
tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputing', KNNImputer())]),
                                 ['time', 'accelerometer_mean',
                                  'accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'linear_acceleration_mean',
                                  'linear_acceleration_min',
                                  'linear_acceleration_max',
                                  'linear_acceleration_std', 'pressure_mean',
                                  'proximity_std', 'rotation_vector_mean',
                                  'rotation_vector_min', 'rotation_vector_max',
                                  'rotation_vector_std', 'step_counter_mean',
                                  'step_counter_max', 'step_counter_min',
                                  'step_counter_std', 'sound_mean'])])

In [136]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [137]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputing',
                                                                   KNNImputer())]),
                                                  ['time', 'accelerometer_mean',
                                                   'accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'linear_acceleration_mean',
                                                   'linear_acceleration_min',
                                                   'linear_acceleration_max',
                                                   'linear_acceleration_std',
                                                   'pressure_mean',
                                                   'proximity_std',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_min',
                                                   'rotation_vector_max',
                                                   'rotation_vector_std',
                                                   'step_counter_mean',
                                                   'step_counter_max',
                                                   'step_counter_min',
                                                   'step_counter_std',
                                                   'sound_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [138]:
#ordinal encoding of the target feature
dataset_test['target_enc']=pd.factorize(dataset_test['target'].tolist())[0]
dataset_train['target_enc']=pd.factorize(dataset_train['target'].tolist())[0]

In [139]:
# split 
x_train = dataset_train.drop(['target','target_enc'], axis=1)
y_train = dataset_train.target
x_val = dataset_test.drop(['target','target_enc'], axis=1)
y_val = dataset_test.target

In [ ]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')